In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [8]:
from pytorchcv.model_provider import get_model
model = get_model("xception", pretrained=False)
model = nn.Sequential(*list(model.children())[:-1]) # Remove original output layer

class Pooling(nn.Module):
    def __init__(self):
        super(Pooling, self).__init__()

        self.p1 = nn.AdaptiveAvgPool2d((1,1))
        self.p2 = nn.AdaptiveMaxPool2d((1,1))

    def forward(self, x):
        x1 = self.p1(x)
        x2 = self.p2(x)
        return (x1+x2) * 0.5

model[0].final_block.pool = nn.Sequential(nn.AdaptiveAvgPool2d((1,1)))

class Head(torch.nn.Module):
    def __init__(self, in_f, out_f):
        super(Head, self).__init__()

        self.f = nn.Flatten()
        self.l = nn.Linear(in_f, 512)
        self.d = nn.Dropout(0.5)
        self.o = nn.Linear(512, out_f)
        self.b1 = nn.BatchNorm1d(in_f)
        self.b2 = nn.BatchNorm1d(512)
        self.r = nn.ReLU()

    def forward(self, x):
        x = self.f(x)
        x = self.b1(x)
        x = self.d(x)

        x = self.l(x)
        x = self.r(x)
        x = self.b2(x)
        x = self.d(x)

        out = self.o(x)
        out = nn.Sigmoid()(out)
        return out

class FCN(torch.nn.Module):
    def __init__(self, base, in_f):
        super(FCN, self).__init__()
        self.base = base
        self.h1 = Head(in_f, 1)

    def forward(self, x):
        x = self.base(x)
        return self.h1(x)

    
class FCN2(torch.nn.Module):
    def __init__(self, base):
        super(FCN2, self).__init__()
        self.base = base
        self.l = nn.Linear(2048*8*8, 1)
    def forward(self, x):
        x = self.base(x)
#         print(x.size(-1))
        x = self.l(x)
#         x = nn.Linear(2048*8*8, 1)(x)
#         print(x.size)
        x = nn.Sigmoid()(x)
        return x
def xception2():
    model = get_model("xception", pretrained=False)
    model = nn.Sequential(*list(model.children())[:-1]) # Remove original output layer
    model[0].final_block.pool = nn.Sequential(nn.Flatten())
    model = FCN2(model)
    return model
# net = []
# model = FCN(model, 2048)
# model = model.cuda()
model = xception2()

In [12]:
model

FCN2(
  (base): Sequential(
    (0): Sequential(
      (init_block): XceptionInitBlock(
        (conv1): ConvBlock(
          (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
          (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activ): ReLU(inplace=True)
        )
        (conv2): ConvBlock(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activ): ReLU(inplace=True)
        )
      )
      (stage1): Sequential(
        (unit1): XceptionUnit(
          (identity_conv): ConvBlock(
            (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
            (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (body): Sequential(
            (block1): DwsConvBlock(
              (conv): DwsConv(
             

In [2]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo
from torch.nn import init

In [3]:
import matplotlib.pyplot as plt
import torchvision.utils
import numpy as np
import random
from PIL import Image
import torch
from torch.autograd import Variable
import PIL.ImageOps    
import torch.nn as nn
from torch import optim

In [4]:
import torchvision
from torch.utils.data import DataLoader,Dataset
from torchvision import datasets, transforms, models
from tqdm import tqdm

In [5]:
class resnext50(nn.Module):
    def __init__(self):
        """Load the pretrained ResNet-152 and replace top fc layer."""
        super(resnext50, self).__init__()

        self.resnext50 = models.resnext50_32x4d(pretrained=True)
        self.fc = nn.Sequential(nn.Linear(2048, 1),
                                 nn.Sigmoid())
        self.resnext50.fc = self.fc
    def forward(self, x):
        out = self.resnext50(x)

        return out

In [26]:
model = models.mobilenet_v2(pretrained=True).cuda()
# model.classifier = nn.Sequential(nn.Linear(1280, 1),
#                                  nn.Sigmoid())
# model.cuda()

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /home/tampm/.cache/torch/checkpoints/mobilenet_v2-b0353104.pth


In [17]:
model

MNASNet(
  (layers): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.00029999999999996696, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.00029999999999996696, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (7): BatchNorm2d(16, eps=1e-05, momentum=0.00029999999999996696, affine=True, track_running_stats=True)
    (8): Sequential(
      (0): _InvertedResidual(
        (layers): Sequential(
          (0): Conv2d(16, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(48, eps=1e-05, momentum=0.00029999999999996696, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(48, 

In [6]:
model = models.mnasnet1_0(pretrained=True).cuda()
model.classifier = nn.Sequential(nn.Linear(1280, 1),nn.Sigmoid())

In [41]:
resnext50 = models.resnext50_32x4d(pretrained=True)
resnext50.fc = nn.Sequential(nn.Linear(2048, 1),
                                 nn.Sigmoid())

In [36]:
model=resnext50()

In [42]:
# model = model.cuda()
resnext50 = resnext50.cuda()

In [40]:
model

resnext50(
  (resnext50): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): S

In [6]:
from torchsummary import summary

In [7]:
# summary(model,(3,256,256))
model = model.cuda()
summary(model,(3,256,256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 127, 127]             864
       BatchNorm2d-2         [-1, 32, 127, 127]              64
              ReLU-3         [-1, 32, 127, 127]               0
         ConvBlock-4         [-1, 32, 127, 127]               0
            Conv2d-5         [-1, 64, 125, 125]          18,432
       BatchNorm2d-6         [-1, 64, 125, 125]             128
              ReLU-7         [-1, 64, 125, 125]               0
         ConvBlock-8         [-1, 64, 125, 125]               0
 XceptionInitBlock-9         [-1, 64, 125, 125]               0
           Conv2d-10          [-1, 128, 63, 63]           8,192
      BatchNorm2d-11          [-1, 128, 63, 63]             256
        ConvBlock-12          [-1, 128, 63, 63]               0
           Conv2d-13         [-1, 64, 125, 125]             576
           Conv2d-14        [-1, 128, 1

In [9]:
model = model.cuda()
summary(model,(3,256,256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 127, 127]             864
       BatchNorm2d-2         [-1, 32, 127, 127]              64
              ReLU-3         [-1, 32, 127, 127]               0
         ConvBlock-4         [-1, 32, 127, 127]               0
            Conv2d-5         [-1, 64, 125, 125]          18,432
       BatchNorm2d-6         [-1, 64, 125, 125]             128
              ReLU-7         [-1, 64, 125, 125]               0
         ConvBlock-8         [-1, 64, 125, 125]               0
 XceptionInitBlock-9         [-1, 64, 125, 125]               0
           Conv2d-10          [-1, 128, 63, 63]           8,192
      BatchNorm2d-11          [-1, 128, 63, 63]             256
        ConvBlock-12          [-1, 128, 63, 63]               0
           Conv2d-13         [-1, 64, 125, 125]             576
           Conv2d-14        [-1, 128, 1

In [32]:
i = np.ones((1,3,256,256))/2

In [33]:
model.forward(transforms.ToTensor()(np.array([i])))

ValueError: pic should be 2/3 dimensional. Got 5 dimensions.

In [49]:
train_transforms = transforms.Compose([transforms.Resize(256,256),
                                       transforms.RandomHorizontalFlip(p=0.5),
                                       transforms.RandomApply([
                                           transforms.RandomRotation(5),
                                           transforms.RandomAffine(degrees=5,scale=(0.95,1.05))
                                           ], p=0.5),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                             std=[0.229, 0.224, 0.225])
                                       
                                       ])
train_data = datasets.ImageFolder('/hdd/tam/kaggle/image/test/',       
# train_data = datasets.ImageFolder('/hdd/tam/Celeb-DF/image/test/',  
                    transform=train_transforms)



In [31]:
import glob
lis = glob.glob('/hdd/tam/Celeb-DF/image/test/*/*.jpg')

In [32]:
lis

['/hdd/tam/Celeb-DF/image/test/df/id3_id2_0001_62.jpg',
 '/hdd/tam/Celeb-DF/image/test/df/id2_id9_0008_123.jpg',
 '/hdd/tam/Celeb-DF/image/test/df/id6_id3_0005_37.jpg',
 '/hdd/tam/Celeb-DF/image/test/df/id29_id37_0008_101.jpg',
 '/hdd/tam/Celeb-DF/image/test/df/id26_id16_0001_67.jpg',
 '/hdd/tam/Celeb-DF/image/test/df/id2_id1_0004_80.jpg',
 '/hdd/tam/Celeb-DF/image/test/df/id37_id3_0006_27.jpg',
 '/hdd/tam/Celeb-DF/image/test/df/id17_id16_0000_12.jpg',
 '/hdd/tam/Celeb-DF/image/test/df/id1_id2_0007_53.jpg',
 '/hdd/tam/Celeb-DF/image/test/df/id2_id9_0008_87.jpg',
 '/hdd/tam/Celeb-DF/image/test/df/id10_id13_0007_58.jpg',
 '/hdd/tam/Celeb-DF/image/test/df/id28_id35_0008_49.jpg',
 '/hdd/tam/Celeb-DF/image/test/df/id25_id19_0008_26.jpg',
 '/hdd/tam/Celeb-DF/image/test/df/id17_id23_0000_42.jpg',
 '/hdd/tam/Celeb-DF/image/test/df/id30_id26_0004_42.jpg',
 '/hdd/tam/Celeb-DF/image/test/df/id46_id41_0000_13.jpg',
 '/hdd/tam/Celeb-DF/image/test/df/id3_id28_0001_43.jpg',
 '/hdd/tam/Celeb-DF/image/

In [33]:
import cv2

In [34]:
for i in lis:
    img = cv2.imread(i)
    print(img.shape)

(266, 187, 3)
(457, 315, 3)
(246, 180, 3)
(233, 149, 3)
(155, 119, 3)
(229, 170, 3)
(335, 253, 3)
(219, 166, 3)
(251, 197, 3)
(454, 310, 3)
(327, 240, 3)
(219, 163, 3)
(204, 147, 3)
(222, 168, 3)
(184, 153, 3)
(168, 126, 3)
(254, 173, 3)
(217, 163, 3)
(239, 170, 3)
(238, 183, 3)
(203, 139, 3)
(253, 188, 3)
(204, 146, 3)
(219, 161, 3)
(254, 187, 3)
(252, 183, 3)
(243, 181, 3)
(245, 181, 3)
(330, 245, 3)
(273, 210, 3)
(237, 166, 3)
(229, 170, 3)
(191, 141, 3)
(209, 146, 3)
(264, 183, 3)
(289, 224, 3)
(209, 154, 3)
(215, 166, 3)
(238, 174, 3)
(153, 121, 3)
(195, 142, 3)
(187, 135, 3)
(229, 166, 3)
(362, 239, 3)
(237, 166, 3)
(156, 126, 3)
(448, 317, 3)
(359, 259, 3)
(243, 181, 3)
(288, 203, 3)
(254, 182, 3)
(261, 204, 3)
(260, 184, 3)
(314, 215, 3)
(351, 254, 3)
(240, 183, 3)
(364, 267, 3)
(190, 145, 3)
(159, 113, 3)
(259, 182, 3)
(170, 114, 3)
(203, 146, 3)
(232, 163, 3)
(253, 187, 3)
(187, 138, 3)
(240, 175, 3)
(216, 148, 3)
(343, 246, 3)
(230, 156, 3)
(194, 159, 3)
(247, 194, 3)
(211, 

(155, 114, 3)
(373, 302, 3)
(253, 181, 3)
(272, 211, 3)
(246, 181, 3)
(250, 196, 3)
(308, 226, 3)
(209, 169, 3)
(384, 301, 3)
(202, 148, 3)
(328, 232, 3)
(288, 198, 3)
(182, 139, 3)
(254, 181, 3)
(177, 146, 3)
(182, 138, 3)
(240, 173, 3)
(250, 194, 3)
(243, 166, 3)
(240, 174, 3)
(216, 149, 3)
(321, 266, 3)
(250, 169, 3)
(287, 209, 3)
(316, 230, 3)
(443, 314, 3)
(191, 138, 3)
(205, 148, 3)
(358, 257, 3)
(187, 125, 3)
(225, 155, 3)
(203, 141, 3)
(343, 252, 3)
(292, 217, 3)
(224, 167, 3)
(197, 153, 3)
(273, 215, 3)
(224, 162, 3)
(226, 170, 3)
(217, 161, 3)
(140, 104, 3)
(252, 176, 3)
(310, 224, 3)
(197, 154, 3)
(217, 154, 3)
(194, 133, 3)
(226, 176, 3)
(237, 153, 3)
(191, 128, 3)
(223, 169, 3)
(254, 182, 3)
(240, 187, 3)
(195, 148, 3)
(280, 208, 3)
(260, 184, 3)
(225, 166, 3)
(215, 155, 3)
(252, 198, 3)
(217, 154, 3)
(266, 187, 3)
(328, 259, 3)
(219, 167, 3)
(229, 170, 3)
(237, 190, 3)
(266, 190, 3)
(429, 317, 3)
(231, 166, 3)
(327, 236, 3)
(307, 204, 3)
(181, 149, 3)
(202, 149, 3)
(196, 

(264, 198, 3)
(250, 184, 3)
(264, 189, 3)
(300, 211, 3)
(398, 281, 3)
(247, 180, 3)
(237, 174, 3)
(317, 243, 3)
(257, 198, 3)
(225, 162, 3)
(196, 146, 3)
(191, 141, 3)
(198, 125, 3)
(194, 142, 3)
(295, 215, 3)
(246, 182, 3)
(331, 243, 3)
(224, 163, 3)
(215, 147, 3)
(440, 295, 3)
(160, 121, 3)
(226, 169, 3)
(422, 302, 3)
(306, 215, 3)
(285, 212, 3)
(265, 189, 3)
(233, 163, 3)
(230, 167, 3)
(240, 180, 3)
(285, 209, 3)
(344, 246, 3)
(260, 205, 3)
(245, 176, 3)
(254, 177, 3)
(232, 167, 3)
(264, 177, 3)
(183, 135, 3)
(236, 160, 3)
(250, 190, 3)
(204, 155, 3)
(246, 176, 3)
(253, 188, 3)
(174, 128, 3)
(300, 243, 3)
(187, 126, 3)
(310, 236, 3)
(464, 293, 3)
(224, 166, 3)
(145, 106, 3)
(218, 161, 3)
(279, 209, 3)
(181, 133, 3)
(244, 180, 3)
(230, 166, 3)
(232, 169, 3)
(153, 114, 3)
(246, 181, 3)
(250, 182, 3)
(233, 181, 3)
(250, 195, 3)
(181, 141, 3)
(259, 196, 3)
(253, 188, 3)
(219, 169, 3)
(219, 161, 3)
(378, 261, 3)
(174, 131, 3)
(238, 181, 3)
(232, 163, 3)
(152, 114, 3)
(237, 163, 3)
(201, 

(210, 153, 3)
(274, 191, 3)
(257, 188, 3)
(314, 219, 3)
(243, 184, 3)
(222, 154, 3)
(247, 188, 3)
(231, 163, 3)
(294, 212, 3)
(243, 183, 3)
(208, 156, 3)
(223, 155, 3)
(237, 175, 3)
(246, 170, 3)
(212, 159, 3)
(247, 181, 3)
(303, 250, 3)
(236, 175, 3)
(195, 155, 3)
(250, 181, 3)
(239, 173, 3)
(149, 104, 3)
(310, 224, 3)
(205, 159, 3)
(229, 176, 3)
(385, 295, 3)
(344, 266, 3)
(246, 182, 3)
(274, 203, 3)
(231, 166, 3)
(287, 209, 3)
(224, 162, 3)
(299, 237, 3)
(254, 182, 3)
(201, 149, 3)
(273, 182, 3)
(244, 175, 3)
(204, 147, 3)
(229, 181, 3)
(313, 243, 3)
(208, 160, 3)
(233, 175, 3)
(246, 180, 3)
(229, 167, 3)
(331, 252, 3)
(154, 113, 3)
(279, 203, 3)
(259, 168, 3)
(240, 166, 3)
(247, 191, 3)
(232, 168, 3)
(149, 111, 3)
(155, 117, 3)
(161, 124, 3)
(352, 265, 3)
(218, 161, 3)
(250, 189, 3)
(253, 197, 3)
(203, 160, 3)
(265, 189, 3)
(313, 237, 3)
(181, 142, 3)
(230, 163, 3)
(198, 147, 3)
(216, 166, 3)
(226, 175, 3)
(252, 187, 3)
(225, 166, 3)
(251, 180, 3)
(211, 145, 3)
(237, 174, 3)
(245, 

(210, 162, 3)
(377, 266, 3)
(217, 148, 3)
(265, 189, 3)
(252, 189, 3)
(114, 89, 3)
(306, 209, 3)
(222, 154, 3)
(219, 166, 3)
(148, 110, 3)
(187, 134, 3)
(243, 177, 3)
(244, 175, 3)
(254, 182, 3)
(244, 194, 3)
(324, 253, 3)
(250, 184, 3)
(149, 110, 3)
(300, 246, 3)
(252, 184, 3)
(240, 177, 3)
(251, 167, 3)
(232, 169, 3)
(338, 250, 3)
(176, 134, 3)
(226, 162, 3)
(233, 163, 3)
(127, 93, 3)
(204, 163, 3)
(222, 163, 3)
(141, 104, 3)
(236, 170, 3)
(245, 173, 3)
(209, 145, 3)
(226, 161, 3)
(343, 252, 3)
(215, 167, 3)
(322, 231, 3)
(245, 166, 3)
(246, 166, 3)
(233, 174, 3)
(229, 174, 3)
(237, 167, 3)
(253, 190, 3)
(279, 202, 3)
(341, 253, 3)
(177, 133, 3)
(271, 196, 3)
(236, 168, 3)
(285, 203, 3)
(289, 212, 3)
(321, 237, 3)
(177, 149, 3)
(296, 224, 3)
(254, 175, 3)
(218, 166, 3)
(352, 246, 3)
(246, 184, 3)
(202, 142, 3)
(334, 267, 3)
(183, 156, 3)
(245, 168, 3)
(216, 163, 3)
(243, 177, 3)
(308, 211, 3)
(282, 202, 3)
(240, 174, 3)
(449, 314, 3)
(342, 253, 3)
(384, 303, 3)
(152, 113, 3)
(308, 22

KeyboardInterrupt: 

In [36]:
# https://discuss.pytorch.org/t/balanced-sampling-between-classes-with-torchvision-dataloader/2703/3
def make_weights_for_balanced_classes(images, nclasses):                        
    count = [0] * nclasses                                                      
    for item in images:                                                         
        count[item[1]] += 1                                                     
    weight_per_class = [0.] * nclasses                                      
    N = float(sum(count))  
    print(count)
    for i in range(nclasses):                                                   
        weight_per_class[i] = N/float(count[i])   
    print(weight_per_class)
    weight = [0] * len(images)                                              
    for idx, val in enumerate(images):                                          
        weight[idx] = weight_per_class[val[1]]                                  
    return weight   
weights = make_weights_for_balanced_classes(train_data.imgs, len(train_data.classes))                                                                
weights = torch.DoubleTensor(weights)                                       
sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights)) 

[31855, 17508]
[1.5496154449850887, 2.819453963902216]


In [37]:
sampler.weights

tensor([1.5496, 1.5496, 1.5496,  ..., 2.8195, 2.8195, 2.8195],
       dtype=torch.float64)

In [50]:
trainloader = torch.utils.data.DataLoader(train_data, batch_size=16,                             
                    sampler = sampler, num_workers=1, pin_memory=True)

In [51]:
iter(trainloader).next()[0].shape

ValueError: Caught ValueError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/tampm/miniconda3/envs/face/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/tampm/miniconda3/envs/face/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/tampm/miniconda3/envs/face/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/tampm/miniconda3/envs/face/lib/python3.7/site-packages/torchvision/datasets/folder.py", line 140, in __getitem__
    sample = self.transform(sample)
  File "/home/tampm/miniconda3/envs/face/lib/python3.7/site-packages/torchvision/transforms/transforms.py", line 70, in __call__
    img = t(img)
  File "/home/tampm/miniconda3/envs/face/li

In [ ]:
test_data = datasets.ImageFolder('/data/tam/kaggle/extract_raw_img_test',       
                    transform=train_transforms)

testloader = torch.utils.data.DataLoader(test_data, batch_size=16,num_workers=1, pin_memory=True)

In [18]:
iter(testloader).next()[0].shape

torch.Size([16, 3, 256, 256])

In [18]:
len(train_data.classes)

2

In [19]:
criterion = nn.BCELoss().cuda()
optimizer = optim.Adam(model.parameters(), lr=0.003)

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() 
                                  else "cpu")

In [30]:
for inputs, labels in tqdm(trainloader):
    pass



  0%|          | 0/39873 [00:00<?, ?it/s]

  0%|          | 1/39873 [00:00<3:13:03,  3.44it/s]

  0%|          | 2/39873 [00:00<2:40:48,  4.13it/s]

  0%|          | 3/39873 [00:00<2:17:02,  4.85it/s]

  0%|          | 4/39873 [00:00<2:00:13,  5.53it/s]

  0%|          | 6/39873 [00:00<1:40:59,  6.58it/s]

  0%|          | 7/39873 [00:00<1:31:51,  7.23it/s]

  0%|          | 8/39873 [00:01<1:31:16,  7.28it/s]

  0%|          | 9/39873 [00:01<2:13:20,  4.98it/s]

  0%|          | 10/39873 [00:01<2:03:16,  5.39it/s]

  0%|          | 12/39873 [00:01<1:49:19,  6.08it/s]

  0%|          | 14/39873 [00:02<1:38:22,  6.75it/s]

  0%|          | 15/39873 [00:02<1:37:11,  6.84it/s]

  0%|          | 16/39873 [00:02<1:30:39,  7.33it/s]

  0%|          | 17/39873 [00:02<1:27:16,  7.61it/s]

  0%|          | 18/39873 [00:02<1:26:08,  7.71it/s]

  0%|          | 20/39873 [00:02<1:19:50,  8.32it/s]

  0%|          | 21/39873 [00:02<1:26:49,  7.65it/s]

  0%|          | 22/39873 [00:02<1:21:24,  8.

  1%|          | 395/39873 [00:43<1:14:54,  8.78it/s]

  1%|          | 396/39873 [00:43<1:18:00,  8.43it/s]

  1%|          | 398/39873 [00:43<1:09:59,  9.40it/s]

  1%|          | 399/39873 [00:44<1:10:55,  9.28it/s]

  1%|          | 400/39873 [00:44<1:16:25,  8.61it/s]

  1%|          | 402/39873 [00:44<1:12:39,  9.05it/s]

  1%|          | 404/39873 [00:44<1:07:28,  9.75it/s]

  1%|          | 406/39873 [00:44<1:02:42, 10.49it/s]

  1%|          | 408/39873 [00:44<1:07:22,  9.76it/s]

  1%|          | 410/39873 [00:45<1:02:48, 10.47it/s]

  1%|          | 412/39873 [00:45<1:10:35,  9.32it/s]

  1%|          | 413/39873 [00:45<1:11:09,  9.24it/s]

  1%|          | 415/39873 [00:45<1:09:21,  9.48it/s]

  1%|          | 416/39873 [00:45<1:11:07,  9.25it/s]

  1%|          | 418/39873 [00:45<1:03:47, 10.31it/s]

  1%|          | 420/39873 [00:46<1:06:35,  9.87it/s]

  1%|          | 422/39873 [00:46<1:06:45,  9.85it/s]

  1%|          | 424/39873 [00:46<1:06:03,  9.95it/s]

  1%|     

  2%|▏         | 861/39873 [01:31<1:17:47,  8.36it/s]

  2%|▏         | 863/39873 [01:31<1:11:10,  9.13it/s]

  2%|▏         | 864/39873 [01:32<1:16:49,  8.46it/s]

  2%|▏         | 865/39873 [01:32<1:17:21,  8.40it/s]

  2%|▏         | 866/39873 [01:32<1:18:45,  8.25it/s]

  2%|▏         | 867/39873 [01:32<1:17:31,  8.39it/s]

  2%|▏         | 868/39873 [01:32<1:14:38,  8.71it/s]

  2%|▏         | 870/39873 [01:32<1:08:01,  9.55it/s]

  2%|▏         | 871/39873 [01:32<1:11:13,  9.13it/s]

  2%|▏         | 873/39873 [01:33<1:06:22,  9.79it/s]

  2%|▏         | 875/39873 [01:33<1:11:30,  9.09it/s]

  2%|▏         | 876/39873 [01:33<1:11:42,  9.06it/s]

  2%|▏         | 877/39873 [01:33<1:10:55,  9.16it/s]

  2%|▏         | 879/39873 [01:33<1:11:21,  9.11it/s]

  2%|▏         | 880/39873 [01:33<1:12:36,  8.95it/s]

  2%|▏         | 882/39873 [01:34<1:09:13,  9.39it/s]

  2%|▏         | 884/39873 [01:34<1:05:52,  9.86it/s]

  2%|▏         | 886/39873 [01:34<1:01:32, 10.56it/s]

  2%|▏    

  3%|▎         | 1092/39873 [01:54<57:53, 11.16it/s]  

  3%|▎         | 1094/39873 [01:54<55:20, 11.68it/s]

  3%|▎         | 1096/39873 [01:55<1:00:07, 10.75it/s]

  3%|▎         | 1098/39873 [01:55<1:00:49, 10.62it/s]

  3%|▎         | 1100/39873 [01:55<55:44, 11.59it/s]  

  3%|▎         | 1102/39873 [01:55<59:38, 10.83it/s]

  3%|▎         | 1104/39873 [01:55<58:28, 11.05it/s]

  3%|▎         | 1106/39873 [01:56<1:00:27, 10.69it/s]

  3%|▎         | 1108/39873 [01:56<1:06:19,  9.74it/s]

  3%|▎         | 1110/39873 [01:56<1:07:08,  9.62it/s]

  3%|▎         | 1112/39873 [01:56<1:05:35,  9.85it/s]

  3%|▎         | 1114/39873 [01:56<1:05:29,  9.86it/s]

  3%|▎         | 1116/39873 [01:57<1:05:30,  9.86it/s]

  3%|▎         | 1118/39873 [01:57<1:04:11, 10.06it/s]

  3%|▎         | 1120/39873 [01:57<1:01:49, 10.45it/s]

  3%|▎         | 1122/39873 [01:57<59:29, 10.86it/s]  

  3%|▎         | 1124/39873 [01:57<1:01:34, 10.49it/s]

  3%|▎         | 1126/39873 [01:58<1:03:00, 10.25it/s]

  4%|▍         | 1565/39873 [02:42<1:07:26,  9.47it/s]

  4%|▍         | 1567/39873 [02:42<1:01:35, 10.37it/s]

  4%|▍         | 1569/39873 [02:42<1:01:59, 10.30it/s]

  4%|▍         | 1571/39873 [02:42<1:01:51, 10.32it/s]

  4%|▍         | 1573/39873 [02:42<1:02:42, 10.18it/s]

  4%|▍         | 1575/39873 [02:43<1:03:17, 10.08it/s]

  4%|▍         | 1577/39873 [02:43<1:16:13,  8.37it/s]

  4%|▍         | 1578/39873 [02:43<1:17:17,  8.26it/s]

  4%|▍         | 1580/39873 [02:43<1:14:36,  8.55it/s]

  4%|▍         | 1581/39873 [02:43<1:12:05,  8.85it/s]

  4%|▍         | 1583/39873 [02:44<1:08:20,  9.34it/s]

  4%|▍         | 1585/39873 [02:44<1:09:07,  9.23it/s]

  4%|▍         | 1586/39873 [02:44<1:08:29,  9.32it/s]

  4%|▍         | 1587/39873 [02:44<1:13:19,  8.70it/s]

  4%|▍         | 1589/39873 [02:44<1:12:23,  8.81it/s]

  4%|▍         | 1591/39873 [02:44<1:08:41,  9.29it/s]

  4%|▍         | 1592/39873 [02:45<1:07:48,  9.41it/s]

  4%|▍         | 1594/39873 [02:45<1:05:48,  9.6

KeyboardInterrupt: 

In [48]:
epochs = 1
steps = 0
running_loss = 0
print_every = 10
train_losses, test_losses = [], []
for epoch in range(epochs):
    for inputs, labels in tqdm(trainloader):
#     for inputs, labels in tqdm(testloader):

        steps += 1
#         labels = np.array([labels])
        inputs, labels = inputs.to(device), labels.float().to(device)
#         inputs, labels = inputs.to(device), labels[1].float().to(device)

        optimizer.zero_grad()
        logps = model.forward(inputs)[:,0]
#         loss = criterion(logps, labels)
        loss = F.binary_cross_entropy_with_logits(logps, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device),labels.float().to(device)
                    logps = model.forward(inputs)[:,0]
                    batch_loss = criterion(logps, labels)
                    test_loss += batch_loss.item()
                    equals = labels == (logps >0.5)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model.train()
            
            #######################
#             with torch.no_grad():
#                 for inputs, labels in testloader:
#                     inputs, labels = inputs.to(device),labels.float().to(device)
#                     logps = model.forward(inputs)
#                     batch_loss = criterion(logps, labels)
#                     test_loss += batch_loss.item()
                    
#                     ps = torch.exp(logps)
#                     top_p, top_class = ps.topk(1, dim=1)
#                     equals = top_class == labels.view(*top_class.shape)
#                     accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
#             train_losses.append(running_loss/len(trainloader))
#             test_losses.append(test_loss/len(testloader))                    
#             print(f"Epoch {epoch+1}/{epochs}.. "
#                   f"Train loss: {running_loss/print_every:.3f}.. "
#                   f"Test loss: {test_loss/len(testloader):.3f}.. "
#                   f"Test accuracy: {accuracy/len(testloader):.3f}")
#             running_loss = 0
#             model.train()





  0%|          | 0/39873 [00:00<?, ?it/s]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f0761393a70>
Traceback (most recent call last):
  File "/home/tampm/miniconda3/envs/face/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 961, in __del__
    self._shutdown_workers()
  File "/home/tampm/miniconda3/envs/face/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 941, in _shutdown_workers
    w.join()
  File "/home/tampm/miniconda3/envs/face/lib/python3.7/multiprocessing/process.py", line 138, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process




  0%|          | 1/39873 [00:00<5:12:48,  2.12it/s]



  0%|          | 3/39873 [00:00<3:59:14,  2.78it/s]



  0%|          | 4/39873 [00:00<3:14:06,  3.42it/s]



  0%|          | 5/39873 [00:00<2:37:21,  4.22it/s]



  0%|          | 6/39873 [00:01<2:18:22,  4.80it/s]



  0%|          | 7/39873 [00:

KeyboardInterrupt: 